In [ ]:
%matplotlib inline
import numpy as np
import time
import scipy
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib.colors as mpcol
import src.dab_util as du
import src.signal_gen as sg
import src.tcp_sync as ts
import src.dab_tuning_lib as dt
import pandas as pd
from tqdm import tqdm
reload(sg)
reload(du)
reload(dt)

In [ ]:
use_fft=False

In [ ]:
if use_fft: t1 = ts.UhdSyncMsg(port=47009, packet_size=4*16384, packet_type="f"*16384)
else: t1 = ts.UhdSyncMsg(port=47009, packet_size=4*1, packet_type="f")

In [ ]:
import src.SendDictTcp as sdt

In [ ]:
sender = sdt.SendDictTcp('127.0.0.1', 1112)

In [ ]:
sender.send({"txgain":80})
sender.send({"rxgain":15})
sender.send({"a1":0.8})
sender.send({"a2":0.0})
sender.send({"a3":0.0})
sender.send({"a4":0.0})
sender.send({"a5":0.0})
sender.send({"a6":0.0})
sender.send({"a7":0.0})
sender.send({"a8":0.0})

In [ ]:
t1.has_msg()
t = time.time()
t1.get_msgs(10000)
print( time.time() - t)
t = time.time()
t1.get_msgs(10000)

In [ ]:
reload(dt)
sender.send({"txgain":81})
sender.send({"rxgain":15})
sender.send({"a1":0.8})
sender.send({"a2":0.0})
sender.send({"a3":0.0})
sender.send({"a4":0.0})
sender.send({"a5":0.0})
sender.send({"a6":0.0})
sender.send({"a7":0.0})
sender.send({"a8":0.0})
t1.has_msg()
np.mean(t1.get_msgs(10000))

In [ ]:
reload(dt)

In [ ]:
def measure_sholders(verbose = False, raw_data=False, std_max=0.025):
    for i in range(20):
        try:
            if verbose: print("%d measurement" % i)
            t1.has_msg()
            msgs = t1.get_msgs_fft(20000)
            #sholders = [dt.calc_signal_sholder_ratio(np.array(msg), 8000000) for msg in msgs]
            #sholders = [dt.calc_max_in_freq_range(np.array(msg), 8000000, 1000000, 1900000, debug=False) for msg in msgs]
            #sholders = [dt.calc_mean_in_freq_range(np.array(msg), 8000000, 1000000, 1900000, debug=False) for msg in msgs]
            sholders = np.array(msgs)
            std = np.std(sholders)
            mean = np.mean(sholders)
            std_perc = std/np.abs(mean)
            if verbose == 2:
                print( {"mean":mean, "std":std, "std_perc":std_perc})
            if std_perc > std_max:
                if verbose: print("%.4f std/mean" % (std_perc))
                continue
            else:
                if raw_data: return sholders
                else: return np.mean(sholders)
        except Exception as e:
            print e
    raise Exception("Variance of measurement to high")


In [ ]:
a = np.array(measure_sholders(verbose=2, std_max=100, raw_data=True)).flatten()

In [ ]:
b = a[a<0.04]
plt.hist(b, bins=100);

In [ ]:
from tqdm import tqdm_notebook

sender.send({"txgain":83})
sender.send({"rxgain":15})
sender.send({"a1":0.8})

res = []
err = []
t = time.time()
for a2 in tqdm_notebook(np.linspace(-0.5, 0.5, num=10)):
    for a3 in tqdm_notebook([0]):
        for a4 in tqdm_notebook([0]):
#for a2 in tqdm_notebook(np.linspace(-0.5, 0.5, num=21)):
#    for a3 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
#        for a4 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
            try:
                sender.send({"a2":a2})
                sender.send({"a3":a3})
                sender.send({"a4":a4})
                losses = measure_sholders(verbose=0, std_max=100, raw_data=True)
                for i, loss in enumerate(losses):
                    res.append({"a2":a2, "a3":a3, "a4":a4, "loss":loss, "run":i})
            except Exception as e:
                err.append(e)
        df = pd.DataFrame(res)
        df.to_csv("/tmp/sholder_a2.csv")
print(time.time() - t)

In [ ]:
res = []
err = []
t = time.time()
for a2 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
    for a3 in tqdm_notebook(np.linspace(-0.5, 0.5, num=21)):
        for a4 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
            try:
                sender.send({"a2":a2})
                sender.send({"a3":a3})
                sender.send({"a4":a4})
                losses = measure_sholders(raw_data=True, std=0.03)
                for i, loss in enumerate(losses):
                    res.append({"a2":a2, "a3":a3, "a4":a4, "loss":loss, "run":i})
            except Exception as e:
                err.append(e)
        df = pd.DataFrame(res)
        df.to_csv("/tmp/sholder_a3.csv")
print(time.time() - t)

In [ ]:
res = []
err = []
t = time.time()
for a2 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
    for a3 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
        for a4 in tqdm_notebook(np.linspace(-0.5, 0.5, num=21)):
            try:
                sender.send({"a2":a2})
                sender.send({"a3":a3})
                sender.send({"a4":a4})
                losses = measure_sholders(raw_data=True, std=0.03)
                for i, loss in enumerate(losses):
                    res.append({"a2":a2, "a3":a3, "a4":a4, "loss":loss, "run":i})
            except Exception as e:
                err.append(e)
        df = pd.DataFrame(res)
        df.to_csv("/tmp/sholder_a4.csv")
print(time.time() - t)

In [ ]:
from tqdm import tqdm_notebook

sender.send({"txgain":81})
sender.send({"rxgain":15})
sender.send({"a1":0.8})

res = []
err = []
t = time.time()
for a2 in tqdm_notebook(np.linspace(-0.5, 0.5, num=21)):
    for a3 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
        for a4 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
            try:
                sender.send({"a2":a2})
                sender.send({"a3":a3})
                sender.send({"a4":a4})
                losses = measure_sholders(raw_data=True, std=0.03)
                for i, loss in enumerate(losses):
                    res.append({"a2":a2, "a3":a3, "a4":a4, "loss":loss, "run":i})
            except Exception as e:
                err.append(e)
        df = pd.DataFrame(res)
        df.to_csv("/tmp/sholder_a2_81.csv")
print(time.time() - t)

res = []
err = []
t = time.time()
for a2 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
    for a3 in tqdm_notebook(np.linspace(-0.5, 0.5, num=21)):
        for a4 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
            try:
                sender.send({"a2":a2})
                sender.send({"a3":a3})
                sender.send({"a4":a4})
                losses = measure_sholders(raw_data=True, std=0.03)
                for i, loss in enumerate(losses):
                    res.append({"a2":a2, "a3":a3, "a4":a4, "loss":loss, "run":i})
            except Exception as e:
                err.append(e)
        df = pd.DataFrame(res)
        df.to_csv("/tmp/sholder_a3_81.csv")
print(time.time() - t)
res = []
err = []
t = time.time()
for a2 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
    for a3 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
        for a4 in tqdm_notebook(np.linspace(-0.5, 0.5, num=21)):
            try:
                sender.send({"a2":a2})
                sender.send({"a3":a3})
                sender.send({"a4":a4})
                losses = measure_sholders(raw_data=True, std=0.03)
                for i, loss in enumerate(losses):
                    res.append({"a2":a2, "a3":a3, "a4":a4, "loss":loss, "run":i})
            except Exception as e:
                err.append(e)
        df = pd.DataFrame(res)
        df.to_csv("/tmp/sholder_a4_81.csv")
print(time.time() - t)

In [ ]:
from tqdm import tqdm_notebook

sender.send({"txgain":85})
sender.send({"rxgain":15})
sender.send({"a1":0.8})

res = []
err = []
t = time.time()
for a2 in tqdm_notebook(np.linspace(-0.5, 0.5, num=21)):
    for a3 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
        for a4 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
            try:
                sender.send({"a2":a2})
                sender.send({"a3":a3})
                sender.send({"a4":a4})
                losses = measure_sholders(raw_data=True, std=0.03)
                for i, loss in enumerate(losses):
                    res.append({"a2":a2, "a3":a3, "a4":a4, "loss":loss, "run":i})
            except Exception as e:
                err.append(e)
        df = pd.DataFrame(res)
        df.to_csv("/tmp/sholder_a2_85.csv")
print(time.time() - t)

res = []
err = []
t = time.time()
for a2 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
    for a3 in tqdm_notebook(np.linspace(-0.5, 0.5, num=21)):
        for a4 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
            try:
                sender.send({"a2":a2})
                sender.send({"a3":a3})
                sender.send({"a4":a4})
                losses = measure_sholders(raw_data=True, std=0.03)
                for i, loss in enumerate(losses):
                    res.append({"a2":a2, "a3":a3, "a4":a4, "loss":loss, "run":i})
            except Exception as e:
                err.append(e)
        df = pd.DataFrame(res)
        df.to_csv("/tmp/sholder_a3_85.csv")
print(time.time() - t)
res = []
err = []
t = time.time()
for a2 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
    for a3 in tqdm_notebook(np.linspace(-0.2, 0.2, num=5)):
        for a4 in tqdm_notebook(np.linspace(-0.5, 0.5, num=21)):
            try:
                sender.send({"a2":a2})
                sender.send({"a3":a3})
                sender.send({"a4":a4})
                losses = measure_sholders(raw_data=True, std=0.03)
                for i, loss in enumerate(losses):
                    res.append({"a2":a2, "a3":a3, "a4":a4, "loss":loss, "run":i})
            except Exception as e:
                err.append(e)
        df = pd.DataFrame(res)
        df.to_csv("/tmp/sholder_a4_85.csv")
print(time.time() - t)

In [ ]:
sender.send({"txgain":20})
sender.send({"rxgain":15})
sender.send({"a1":0.1})